# Generate Reactions

This script performs the same task as the script in `scripts/generateReactions.py` but in visual ipynb format.
It can also evaluate the reaction forward and reverse rates at a user selected temperature.

In [ ]:
from rmgpy.rmg.main import RMG
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy import settings
from IPython.display import display
from rmgpy.cantherm.output import prettify

Declare database variables here by changing the thermo and reaction libraries, or restrict to certain reaction families.  

In [ ]:
database = """
database(
    thermoLibraries = ['KlippensteinH2O2','SulfurLibrary', 'primaryThermoLibrary','DFT_QCI_thermo','CBS_QB3_1dHR'],
    reactionLibraries = [],  
    seedMechanisms = [],
    kineticsDepositories = 'default', 
    kineticsFamilies = ['Intra_R_Add_Exocyclic'],  # Select a few families
#   kineticsFamilies = 'all',   # Or select 'all' or 'default' for the families
    kineticsEstimator = 'rate rules',
)

options(
    verboseComments=True,  # Set to True for detailed kinetics comments
)
"""

List all species you want reactions between

In [ ]:
speciesList = """
species(
    label = "RAD1",
    structure = SMILES("CCCCCCCCCCCCc1[c]cccc1"))
    
species(
    label = "RAD1",
    structure = SMILES("CCCCCCCCCCC[CH]c1ccccc1"))

species(
    label = "RAD2",
    structure = SMILES("CCCCCCCCCC[CH]Cc1ccccc1"))

species(
    label = "RAD3",
    structure = SMILES("CCCCCCCCC[CH]CCc1ccccc1"))

species(
    label = "RAD4",
    structure = SMILES("CCCCCCCC[CH]CCCc1ccccc1"))

species(
    label = "RAD5",
    structure = SMILES("CCCCCCC[CH]CCCCc1ccccc1"))

species(
    label = "RAD6",
    structure = SMILES("CCCCCC[CH]CCCCCc1ccccc1"))

species(
    label = "RAD7",
    structure = SMILES("CCCCC[CH]CCCCCCc1ccccc1"))

species(
    label = "RAD8",
    structure = SMILES("CCCC[CH]CCCCCCCc1ccccc1"))

species(
    label = "RAD9",
    structure = SMILES("CCC[CH]CCCCCCCCc1ccccc1"))

species(
    label = "RAD10",
    structure = SMILES("CC[CH]CCCCCCCCCc1ccccc1"))

species(
    label = "RAD11",
    structure = SMILES("C[CH]CCCCCCCCCCc1ccccc1"))

species(
    label = "RAD12",
    structure = SMILES("[CH2]CCCCCCCCCCCc1ccccc1"))
"""

In [ ]:
# Write input file to disk
inputFile = open('temp/input.py','w')
inputFile.write(database)
inputFile.write(speciesList)
inputFile.close()

In [ ]:
# Execute generate reactions
from rmgpy.tools.generate_reactions import *
rmg = RMG(inputFile='temp/input.py', outputDirectory='temp')
rmg = execute(rmg)

In [ ]:
# Pick some temperature to evaluate the forward and reverse kinetics
T = 623.0 # K

In [ ]:
for rxn in rmg.reactionModel.outputReactionList:
    print '========================='
    display(rxn)
    print 'Reaction Family = {0}'.format(rxn.family)
    print ''
    print 'Reactants'
    for reactant in rxn.reactants:
        print 'Label: {0}'.format(reactant.label)
        print 'SMILES: {0}'.format(reactant.molecule[0].toSMILES())
        print ''
    print 'Products'
    for product in rxn.products:
        print 'Label: {0}'.format(product.label)
        print 'SMILES: {0}'.format(product.molecule[0].toSMILES())
    print ''
    print rxn.toChemkin()
    print ''
    print 'Heat of Reaction = {0:.2F} kcal/mol'.format(rxn.getEnthalpyOfReaction(623.0)/4184)
    print 'Forward kinetics at {0} K: {1:.2E}'.format(T, rxn.getRateCoefficient(T))

    reverseRate = rxn.generateReverseRateCoefficient()
    print 'Reverse kinetics at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))